In [1]:
from  selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import os
from os.path  import basename
import requests
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import re

In [2]:
# Read airports
airports_df = pd.read_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'airports.csv'))
airports_df.head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [3]:
airports_code = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='Airports')
airports_code.columns = airports_code.iloc[0]
airports_code = airports_code.iloc[1:]
airports_code

,City,FAA,IATA,ICAO,Airport,Role,Enplanements
1,ALABAMA,NaN,NaN,NaN,NaN,NaN,NaN
2,Birmingham,BHM,BHM,KBHM,Birmingham–Shuttlesworth International Airport,P-S,"1,304,467"
3,Dothan,DHN,DHN,KDHN,Dothan Regional Airport,P-N,49.411
4,Huntsville,HSV,HSV,KHSV,Huntsville International Airport (Carl T. Jone...,P-S,527.801
5,Mobile,MOB,MOB,KMOB,Mobile Regional Airport,P-N,288.209
...,...,...,...,...,...,...,...
446,San Juan / Miramar,SIG,SIG,TJIG,Fernando Luis Ribas Dominicci Airport (Isla Gr...,P-N,24.054
447,Vieques,VQS,VQS,TJVQ,Antonio Rivera Rodríguez Airport,P-N,63.104
448,U.S. VIRGIN ISLANDS,NaN,NaN,NaN,NaN,NaN,NaN
449,"Charlotte Amalie, St. Thomas",STT,STT,TIST,Cyril E. King Airport,P-N,729.382


In [4]:
states = pd.read_excel(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'US_airports.xlsx'), sheet_name='States')
states.columns = ['state_name','state_acronym']

In [5]:
airports = pd.merge(airports_code, states, left_on='City', right_on='state_name', how='left')
airports.head()

,City,FAA,IATA,ICAO,Airport,Role,Enplanements,state_name,state_acronym
0,ALABAMA,NaN,NaN,NaN,NaN,NaN,NaN,ALABAMA,AL
1,Birmingham,BHM,BHM,KBHM,Birmingham–Shuttlesworth International Airport,P-S,"1,304,467",NaN,NaN
2,Dothan,DHN,DHN,KDHN,Dothan Regional Airport,P-N,49.411,NaN,NaN
3,Huntsville,HSV,HSV,KHSV,Huntsville International Airport (Carl T. Jone...,P-S,527.801,NaN,NaN
4,Mobile,MOB,MOB,KMOB,Mobile Regional Airport,P-N,288.209,NaN,NaN


In [6]:
airports[['state_name','state_acronym']] = airports[['state_name','state_acronym']].ffill(axis = 0)

In [7]:
airports = airports[~airports['FAA'].isnull()]

In [8]:
airports

,City,FAA,IATA,ICAO,Airport,Role,Enplanements,state_name,state_acronym
1,Birmingham,BHM,BHM,KBHM,Birmingham–Shuttlesworth International Airport,P-S,"1,304,467",ALABAMA,AL
2,Dothan,DHN,DHN,KDHN,Dothan Regional Airport,P-N,49.411,ALABAMA,AL
3,Huntsville,HSV,HSV,KHSV,Huntsville International Airport (Carl T. Jone...,P-S,527.801,ALABAMA,AL
4,Mobile,MOB,MOB,KMOB,Mobile Regional Airport,P-N,288.209,ALABAMA,AL
5,Montgomery,MGM,MGM,KMGM,Montgomery Regional Airport (Dannelly Field),P-N,173.21,ALABAMA,AL
...,...,...,...,...,...,...,...,...,...
444,San Juan / Carolina,SJU,SJU,TJSJ,Luis Muñoz Marín International Airport,P-M,"4,343,354",PUERTO RICO,PR
445,San Juan / Miramar,SIG,SIG,TJIG,Fernando Luis Ribas Dominicci Airport (Isla Gr...,P-N,24.054,PUERTO RICO,PR
446,Vieques,VQS,VQS,TJVQ,Antonio Rivera Rodríguez Airport,P-N,63.104,PUERTO RICO,PR
448,"Charlotte Amalie, St. Thomas",STT,STT,TIST,Cyril E. King Airport,P-N,729.382,PUERTO RICO,PR


In [82]:
airports.to_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/airlines_airports', 'airports_full.csv'))

In [9]:
main_link = 'https://www.wunderground.com/history'

In [76]:
driver = webdriver.Chrome(ChromeDriverManager().install())



====== WebDriver manager ======
Current google-chrome version is 95.0.4638
Get LATEST driver version for 95.0.4638
Driver [/Users/carlosperezricardo/.wdm/drivers/chromedriver/mac64/95.0.4638.54/chromedriver] found in cache


In [77]:
driver.get(main_link)
time.sleep(3)

In [78]:
# Cookies
try:
    driver.find_element_by_id("truste-consent-button").click()
    time.sleep(1)
except:
    pass

In [63]:
# Send ICAO code
driver.find_element_by_name("historySearch").send_keys("KHSV")
time.sleep(5)

In [64]:
# Click view
#submit-btn button radius ng-star-inserted
driver.find_element_by_class_name("submit-btn").click()
time.sleep(2)

driver.find_element_by_class_name("submit-btn").click()
time.sleep(1)

In [65]:
get_url = driver.current_url
get_url

'https://www.wunderground.com/history/daily/KHSV/date/2021-11-1'

In [66]:
# We change url 
base_url = '/'.join(get_url.replace('daily','monthly').split('/')[:-1])
base_url

'https://www.wunderground.com/history/monthly/KHSV/date'

In [67]:
year = 2021
month = 10
new_url = ''.join([base_url, '/', str(year), '-', str(month)])
new_url

'https://www.wunderground.com/history/monthly/KHSV/date/2021-10'

In [68]:
driver.get(new_url)

In [69]:
# Click View button
#driver.find_element_by_id("dateSubmit").click()
time.sleep(4)

In [20]:
# Check if table exists

In [21]:
# Read table
content = driver.page_source
soup = BeautifulSoup(content)

In [22]:
leyenda = ['Temperature','Dew Point','Humidity','Wind Speed','Pressure']
units = ['° F','(° F)','(%)','(mph)','(Hg)','(in)']

In [23]:
filas_leyenda = []
filas_leyenda.append('Day')

for i in leyenda:
    filas_leyenda.append(i+'_Max')
    filas_leyenda.append(i+'_Avg')
    filas_leyenda.append(i+'_Min')

filas_leyenda.append('Precipitation')
filas_leyenda

['Day',
 'Temperature_Max',
 'Temperature_Avg',
 'Temperature_Min',
 'Dew Point_Max',
 'Dew Point_Avg',
 'Dew Point_Min',
 'Humidity_Max',
 'Humidity_Avg',
 'Humidity_Min',
 'Wind Speed_Max',
 'Wind Speed_Avg',
 'Wind Speed_Min',
 'Pressure_Max',
 'Pressure_Avg',
 'Pressure_Min',
 'Precipitation']

In [24]:
content = driver.page_source
soup = BeautifulSoup(content)

tabla = soup.find("table",{"class":"days ng-star-inserted"})

In [25]:
def obtain_row_data (tr, tipo_columna):
    lista_columnas = []

    for td in tr.findAll(tipo_columna):
        lista_columnas.append(td.get_text(strip=True))
    
    return lista_columnas

In [26]:
def obtenDatosTabla(tabla):
    filas_procesadas = []
    filas = tabla.findAll("tr")

    start = False
    for tr in filas:
        res = obtain_row_data(tr, "td")
        #for item in res:
            #try:
            #float(item)
        if len(res) == 3:
            start = True
            
        if start == True:
            filas_procesadas.append( res )
            #print(item)
                
            #except:
            #    pass
        #i += 1
        #if i == 2:
        #    break

    return filas_procesadas

In [27]:
lista_tabla = obtenDatosTabla(tabla)
lista_tabla_copy = lista_tabla.copy()

temperature = pd.DataFrame(lista_tabla[1:32])
lista_tabla_copy = lista_tabla_copy[32:]

dew_point = pd.DataFrame(lista_tabla_copy[1:32])
lista_tabla_copy = lista_tabla_copy[32:]

humidity = pd.DataFrame(lista_tabla_copy[1:32])
lista_tabla_copy = lista_tabla_copy[32:]

wind_speed = pd.DataFrame(lista_tabla_copy[1:32])
lista_tabla_copy = lista_tabla_copy[32:]

pressure = pd.DataFrame(lista_tabla_copy[1:32])
lista_tabla_copy = lista_tabla_copy[32:]

precipitation = pd.DataFrame(lista_tabla_copy[1:32])

In [28]:
days = pd.DataFrame(pd.date_range(start="2021-10-01",end="2021-10-31").astype(str))

In [29]:
add_weather = pd.concat([days, temperature, dew_point, humidity, wind_speed, \
    pressure, precipitation],axis=1)
add_weather.columns = filas_leyenda
add_weather

,Day,Temperature_Max,Temperature_Avg,Temperature_Min,Dew Point_Max,Dew Point_Avg,Dew Point_Min,Humidity_Max,Humidity_Avg,Humidity_Min,Wind Speed_Max,Wind Speed_Avg,Wind Speed_Min,Pressure_Max,Pressure_Avg,Pressure_Min,Precipitation
0,2021-10-01,86,73.6,64,69,65.9,63,96,78.8,49,10,3.9,0,29.5,29.5,29.4,0.00
1,2021-10-02,79,71.9,64,70,67.0,63,97,85.1,72,10,6.1,3,29.5,29.5,29.4,0.00
2,2021-10-03,73,70.6,68,70,68.3,67,96,92.2,84,10,6.4,3,29.4,29.3,29.3,0.15
3,2021-10-04,80,69.3,63,67,65.3,63,100,87.9,56,6,1.8,0,29.3,29.3,29.3,0.20
4,2021-10-05,75,69.8,67,69,66.1,64,97,88.5,71,16,6.5,0,29.4,29.3,29.3,0.01
5,2021-10-06,78,68.7,63,71,66.2,61,97,91.8,76,10,5.0,0,29.4,29.4,29.4,1.37
6,2021-10-07,79,67.5,61,69,64.8,61,100,91.6,64,8,2.6,0,29.5,29.4,29.4,0.00
7,2021-10-08,81,65.4,60,66,61.7,60,100,89.5,54,8,0.9,0,29.5,29.4,29.4,0.00
8,2021-10-09,84,67.3,60,66,62.1,59,100,85.4,46,6,2.7,0,29.4,29.4,29.3,0.00
9,2021-10-10,82,71.1,62,68,65.3,61,97,83.0,60,9,5.0,0,29.4,29.3,29.3,0.00


In [71]:
def read_table(driver, filas_leyenda, days):
    content = driver.page_source
    soup = BeautifulSoup(content)

    def obtain_row_data (tr, tipo_columna):
        lista_columnas = []

        for td in tr.findAll(tipo_columna):
            lista_columnas.append(td.get_text(strip=True))
        
        return lista_columnas

    def obtenDatosTabla(tabla):
        filas_procesadas = []
        filas = tabla.findAll("tr")

        start = False
        for tr in filas:
            res = obtain_row_data(tr, "td")
            if len(res) == 3:
                start = True       
            if start == True:
                filas_procesadas.append( res )

        return filas_procesadas    

    lista_tabla = obtenDatosTabla(tabla)
    lista_tabla_copy = lista_tabla.copy()

    temperature = pd.DataFrame(lista_tabla[1:32])
    lista_tabla_copy = lista_tabla_copy[32:]

    dew_point = pd.DataFrame(lista_tabla_copy[1:32])
    lista_tabla_copy = lista_tabla_copy[32:]

    humidity = pd.DataFrame(lista_tabla_copy[1:32])
    lista_tabla_copy = lista_tabla_copy[32:]

    wind_speed = pd.DataFrame(lista_tabla_copy[1:32])
    lista_tabla_copy = lista_tabla_copy[32:]

    pressure = pd.DataFrame(lista_tabla_copy[1:32])
    lista_tabla_copy = lista_tabla_copy[32:]

    precipitation = pd.DataFrame(lista_tabla_copy[1:32])

    add_weather = pd.concat([pd.DataFrame(days), temperature, dew_point, humidity, wind_speed, \
    pressure, precipitation],axis=1)
    add_weather.columns = filas_leyenda
    
    return add_weather

In [30]:
#weather = pd.DataFrame()
#weather = pd.concat([weather, add_weather], axis=0, ignore_index=True)

In [79]:
# For airports
# For years 
# For months 
weather = pd.DataFrame()

for airport in airports['ICAO']:

    # Main link
    driver.get(main_link)
    time.sleep(3)

    driver.find_element_by_name("historySearch").send_keys(airport)
    time.sleep(4)

    # Click view
    #submit-btn button radius ng-star-inserted
    driver.find_element_by_class_name("submit-btn").click()
    time.sleep(2)

    driver.find_element_by_class_name("submit-btn").click()
    time.sleep(1)

    get_url = driver.current_url
    get_url

    # We change url 
    base_url = '/'.join(get_url.replace('daily','monthly').split('/')[:-1])
    base_url

    for year in [2019, 2020, 2021]:
        for month in range(1,13):
                new_url = ''.join([base_url, '/', str(year), '-', str(month)])

                num_days = calendar.monthrange(year, month)[1]
                days = [datetime.date(year, month, day).strftime('%Y-%m-%d') for day in range(1, num_days+1)]
                days

                driver.get(new_url)
                time.sleep(4)

                add_weather = read_table(driver, filas_leyenda, days)
                add_weather['ICAO'] = airport

                weather = pd.concat([weather, add_weather], axis=0, ignore_index=True)
                #print(weather.shape)
        print(year)
        weather.to_csv(os.path.join('/Users/carlosperezricardo/Desktop/TFM/weather', 'weather.csv'))
    print(airport)

2019
2020
2021
KBHM
2019
2020
2021
KDHN
2019
2020
2021
KHSV
2019
2020
2021
KMOB
2019
2020
2021
KMGM
2019
2020
2021
PALH
2019
2020
2021
PAMR
2019
2020
2021
PANC
2019
2020
2021
PANI
2019
2020
2021
PABE


TimeoutException: Message: timeout: Timed out receiving message from renderer: 300.000
  (Session info: chrome=95.0.4638.54)


In [45]:
weather

""


In [37]:
import datetime, calendar
year = 2014
month = 1


['2014-01-01',
 '2014-01-02',
 '2014-01-03',
 '2014-01-04',
 '2014-01-05',
 '2014-01-06',
 '2014-01-07',
 '2014-01-08',
 '2014-01-09',
 '2014-01-10',
 '2014-01-11',
 '2014-01-12',
 '2014-01-13',
 '2014-01-14',
 '2014-01-15',
 '2014-01-16',
 '2014-01-17',
 '2014-01-18',
 '2014-01-19',
 '2014-01-20',
 '2014-01-21',
 '2014-01-22',
 '2014-01-23',
 '2014-01-24',
 '2014-01-25',
 '2014-01-26',
 '2014-01-27',
 '2014-01-28',
 '2014-01-29',
 '2014-01-30',
 '2014-01-31']

In [31]:
# Select history


In [32]:
# Select monthly


In [33]:
https://www.wunderground.com/history/monthly/us/dc/washington/KDCA

SyntaxError: invalid syntax (<ipython-input-33-6289dcc92f8f>, line 1)